Preprocess and clean Job, Health, and Personality EMA data from S3 bucket (in MGT)

In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [2]:
dir_path_MGT = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/Data_googledrive/MGT'

In [3]:
#get list of path names in MGT directory
path_names = []
files = os.listdir(dir_path_MGT)
for file in files: 
        path_names.append(dir_path_MGT + "/" + file)

len(path_names)

346

In [4]:
#three different types of surveys contained in MGT, select example of each to examine further
job_expl = path_names[0]
health_expl = path_names[1]
personality_expl = path_names[3]

In [5]:
#read each example into df
job_expl_df = pd.read_csv(job_expl)
job_expl_df = pd.DataFrame(data = job_expl_df)
job_expl_df_columns = job_expl_df.columns.values
print('job_expl_df', job_expl_df.shape, '\n')

health_expl_df = pd.read_csv(health_expl)
health_expl_df = pd.DataFrame(data = health_expl_df)
health_expl_df_columns = health_expl_df.columns.values
print('health_expl_df', health_expl_df.shape, '\n')

personality_expl_df = pd.read_csv(personality_expl)
personality_expl_df = pd.DataFrame(data = personality_expl_df)
personality_expl_df_columns = personality_expl_df.columns.values
print('personality_expl_df', personality_expl_df.shape, '\n')

job_expl_df (26, 103) 

health_expl_df (27, 99) 

personality_expl_df (29, 74) 



In [6]:
#each survey had a set of common questions asked every time and then a set of questions specific to that survey type

shared_cols = ['V1', 'V2', 'V3', 'V4', 'V5', 'Name', 'Date', 'DayWeek',
       'surveytype', 'Timestamp', 'Timesent', 'Q_TotalDuration',
       'context1', 'Context1Time_1', 'Context1Time_2', 'Context1Time_3',
       'Context1Time_4', 'context2', 'context2_TEXT', 'Context2Time_1',
       'Context2Time_2', 'Context2Time_3', 'Context2Time_4', 'context3',
       'context3_TEXT', 'Context3Time_1', 'Context3Time_2',
       'Context3Time_3', 'Context3Time_4', 'context4', 'context4_TEXT',
       'Context4Time_1', 'Context4Time_2', 'Context4Time_3',
       'Context4Time_4', 'stress', 'StressTime_1', 'StressTime_2',
       'StressTime_3', 'StressTime_4', 'anxiety', 'AnxietyTime_1',
       'AnxietyTime_2', 'AnxietyTime_3', 'AnxietyTime_4', 'pand1',
       'pand2', 'pand3', 'pand4', 'pand5', 'pand6', 'pand7', 'pand8',
       'pand9', 'pand10', 'PANASTime_1', 'PANASTime_2', 'PANASTime_3',
       'PANASTime_4']

personality_only_cols = ['bfid1', 'bfid2', 'bfid3', 'bfid4', 'bfid5',
       'bfid6', 'bfid7', 'bfid8', 'bfid9', 'bfid10', 'bfidTime_1',
       'bfidTime_2', 'bfidTime_3', 'bfidTime_4', 'Unnamed: 73']

health_only_cols = ['sleep_1', 'sleepTime_1', 'sleepTime_2',
       'sleepTime_3', 'sleepTime_4', 'ex1_1', 'ex2_1', 'exTime_1',
       'exTime_2', 'exTime_3', 'exTime_4', 'tob1', 'tob1Time_1',
       'tob1Time_2', 'tob1Time_3', 'tob1Time_4', 'tob2_1', 'tob2_2',
       'tob2_3', 'tob2_4', 'tob2_5', 'tob2_6', 'tob2_7', 'tob2Time_1',
       'tob2Time_2', 'tob2Time_3', 'tob2Time_4', 'alc1', 'alc1Time_1',
       'alc1Time_2', 'alc1Time_3', 'alc1Time_4', 'alc2_1', 'alc2_2',
       'alc2_3', 'alc2Time_1', 'alc2Time_2', 'alc2Time_3', 'alc2Time_4',
       'Unnamed: 98']

job_only_cols = ['work', 'WorkTime_1', 'WorkTime_2', 'WorkTime_3',
       'WorkTime_4', 'itpd3', 'itpd1', 'itpd2', 'itpdTime_1',
       'itpdTime_2', 'itpdTime_3', 'itpdTime_4', 'irbd1', 'irbd2',
       'irbd3', 'irbd4', 'irbd5', 'irbd6', 'irbd7', 'irbdTime_1',
       'irbdTime_2', 'irbdTime_3', 'irbdTime_4', 'dalal1', 'dalal2',
       'dalal3', 'dalal4', 'dalal5', 'dalal6', 'dalal7', 'dalal8',
       'dalal9', 'dalal10', 'dalal11', 'dalal12', 'dalal13', 'dalal14',
       'dalal15', 'dalal16', 'dalalTime_1', 'dalalTime_2', 'dalalTime_3',
       'dalalTime_4', 'Unnamed: 102']

unique_cols = shared_cols + job_only_cols + health_only_cols + personality_only_cols 

In [7]:
all_3_expl_df = pd.concat([job_expl_df, health_expl_df, personality_expl_df])
print('unique_cols length', len(unique_cols))
print(all_3_expl_df.shape)

unique_cols length 158
(82, 158)


In [8]:
#create list containing all files
list_of_dfs = [pd.read_csv(filename) for filename in path_names]
#combine into one pandas df
combined_df = pd.concat(list_of_dfs, ignore_index=True)
print(combined_df.shape)

(11846, 265)


In [13]:
combined_df['Name'].value_counts()

Name                   346
SY1004                  84
SY1006                  72
SD1098                  72
SG1048                  72
SD1035                  71
SD1028                  71
SD1019                  71
SD1076                  71
SD1086                  71
SD1109                  71
SD1013                  71
SD1107                  71
SD1040                  71
SD1030                  71
SD1103                  71
SD1089                  70
SD1054                  70
SD1092                  70
SD1099                  70
SD1044                  70
SG1066                  70
SD1023                  69
SD1006                  69
SD1032                  69
SD1095                  69
SD1113                  69
SG1033                  69
SD1123                  69
SD1074                  69
SD1070                  69
SD1131                  69
SD1037                  69
SD1043                  69
SD1048                  69
SD1002                  68
SG1067                  68
S

In [ ]:
#fix columns names
combined_df = combined_df.rename({'V1': 'ResponseID', 'V2': 'ResponseSet', 'V3': 'StartDate', 'V4': 'EndDate', 'V5': 'Finished'}, axis='columns')

In [ ]:
#select out columns related for time 
time_columns = combined_df.filter(like='Time_').columns.values
combined_df_time = combined_df[time_columns]
print(combined_df_time.shape)

combined_df_no_time = combined_df.drop(time_columns, axis=1)
print(combined_df_no_time.shape)

print('does the math add up?', combined_df.shape[1] == combined_df_no_time.shape[1] + combined_df_time.shape[1])

In [ ]:
combined_df_no_time['surveytype'].value_counts()

In [ ]:
#remove rows that do not have one of the three surveytypes listed (most of these are dulicate rows resulting from original pd.concat)
combined_df_no_time = combined_df_no_time[combined_df['surveytype'] != 'surveytype']
combined_df_no_time = combined_df_no_time[combined_df['surveytype'] != '{"ImportId":"surveytype"}']
print(combined_df_no_time.shape)
combined_df_no_time['surveytype'].value_counts()

In [ ]:
#appartent duplicate column names (StartDate, EndDate, Finished), give unique names so can investigate further
combined_df_no_time.columns = ['Date', 'DayWeek', 'DistributionChannel', 'Duration (in seconds)',
       'EndDate1', 'Finished1', 'Name', 'Progress', 'Q_TotalDuration',
       'RecordedDate', 'ResponseId', 'StartDate1', 'Status', 'Timesent',
       'Timestamp', 'Unnamed: 102', 'Unnamed: 73', 'Unnamed: 98',
       'UserLanguage', 'ResponseID', 'ResponseSet', 'StartDate2',
       'EndDate2', 'Finished2', 'alc1', 'alc2_1', 'alc2_2', 'alc2_3',
       'anxiety', 'bfid1', 'bfid10', 'bfid2', 'bfid3', 'bfid4', 'bfid5',
       'bfid6', 'bfid7', 'bfid8', 'bfid9', 'bfid_bfid1', 'bfid_bfid10',
       'bfid_bfid2', 'bfid_bfid3', 'bfid_bfid4', 'bfid_bfid5',
       'bfid_bfid6', 'bfid_bfid7', 'bfid_bfid8', 'bfid_bfid9', 'context1',
       'context2', 'context2_10_TEXT', 'context2_TEXT', 'context3',
       'context3_7_TEXT', 'context3_TEXT', 'context4', 'context4_3_TEXT',
       'context4_TEXT', 'dalal1', 'dalal10', 'dalal11', 'dalal12',
       'dalal13', 'dalal14', 'dalal15', 'dalal16', 'dalal2', 'dalal3',
       'dalal4', 'dalal5', 'dalal6', 'dalal7', 'dalal8', 'dalal9',
       'dalal_dalal1', 'dalal_dalal10', 'dalal_dalal11', 'dalal_dalal12',
       'dalal_dalal13', 'dalal_dalal14', 'dalal_dalal15', 'dalal_dalal16',
       'dalal_dalal2', 'dalal_dalal3', 'dalal_dalal4', 'dalal_dalal5',
       'dalal_dalal6', 'dalal_dalal7', 'dalal_dalal8', 'dalal_dalal9',
       'ex1_1', 'ex2_1', 'irbd1', 'irbd2', 'irbd3', 'irbd4', 'irbd5',
       'irbd6', 'irbd7', 'irbd_irbd1', 'irbd_irbd2', 'irbd_irbd3',
       'irbd_irbd4', 'irbd_irbd5', 'irbd_irbd6', 'irbd_irbd7', 'itpd1',
       'itpd2', 'itpd3', 'itpd_itpd1', 'itpd_itpd2', 'itpd_itpd3',
       'pand1', 'pand10', 'pand2', 'pand3', 'pand4', 'pand5', 'pand6',
       'pand7', 'pand8', 'pand9', 'pand_pand1', 'pand_pand10',
       'pand_pand2', 'pand_pand3', 'pand_pand4', 'pand_pand5',
       'pand_pand6', 'pand_pand7', 'pand_pand8', 'pand_pand9', 'sleep_1',
       'stress', 'surveytype', 'tob1', 'tob2_1', 'tob2_2', 'tob2_3',
       'tob2_4', 'tob2_5', 'tob2_6', 'tob2_7', 'work']

In [ ]:
#create variable of columns that contain meta data
meta_data_cols = ['Date', 'DayWeek', 'DistributionChannel', 'Duration (in seconds)',
       'EndDate1', 'Finished1', 'Name', 'Progress', 'Q_TotalDuration',
       'RecordedDate', 'ResponseId', 'StartDate1', 'Status', 'Timesent',
       'Timestamp', 'Unnamed: 102', 'Unnamed: 73', 'Unnamed: 98',
       'UserLanguage', 'ResponseID', 'ResponseSet', 'StartDate2',
       'EndDate2', 'Finished2', 'surveytype']

In [ ]:
#two different columns corresponding to ResponseID ('ResponseId' and 'ResponseID') with some NaNs in each, need to reconcile and then delete duplicate
combined_df_no_time.loc[combined_df_no_time['ResponseID'].isnull(), 'ResponseID'] = combined_df_no_time.loc[combined_df_no_time['ResponseID'].isnull()]['ResponseId'].values
combined_df_no_time.loc[combined_df_no_time['ResponseId'].isnull(), 'ResponseId'] = combined_df_no_time.loc[combined_df_no_time['ResponseId'].isnull()]['ResponseID'].values

print('does the math add up?', ((combined_df_no_time['ResponseId'] == combined_df_no_time['ResponseID']) == True).sum() == combined_df_no_time.shape[0])

In [ ]:
#explore missing values - focus on columns with metadata only
combined_df_no_time[meta_data_cols].isnull().sum()

In [ ]:
#drop meta param columns with majority null
combined_df_no_time = combined_df_no_time.drop(['DistributionChannel', 
                                                'Duration (in seconds)',
                                                'EndDate1', 
                                                'Finished1', 
                                                'Progress',
                                                'RecordedDate', 
                                                'ResponseId', 
                                                'StartDate1',
                                                'Status',
                                                'Unnamed: 102', 
                                                'Unnamed: 73', 
                                                'Unnamed: 98',
                                                'UserLanguage'], axis = 1)
print(combined_df_no_time.shape)

In [ ]:
#remove 'R_' from ResponseID column
combined_df_no_time['ResponseID'] = combined_df_no_time['ResponseID'].str.lstrip('R_')

In [ ]:
#save this info as csv to use for subsequent analysis and to share with JV and rest of group
combined_df_no_time.to_csv('mgt_final.csv')